In [5]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """    

    network_input, network_output = prepare_sequences(get_notes())
    validate_input, validate_output = prepare_sequences(get_notes("test"))
    model = create_network(network_input)

    train(model, network_input, network_output, validate_input, validate_output)
    
    
def merge_notes(notes_dict, max_offset):
            
    ret = np.array([])
    for i in np.arange(0, max_offset, 0.5):
        pitches = np.zeros(88)
        if i in notes_dict:            
            for element in notes_dict[i]:            
                if isinstance(element, note.Note):
                    pitches[element.pitch.midi-21] = 1
                else:
                    for p in element.pitches:
                        pitches[p.midi-21] = 1    
        ret = np.append(ret, pitches)
    return ret
    

def get_notes(folder="midi_songs"):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = np.array([])

    for file in glob.glob("{}/*.mid".format(folder)):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        notes_dict = {}
        max_offset = 0
        for element in notes_to_parse:        
            if isinstance(element, note.Note) or isinstance(element, chord.Chord):  
                if element.offset not in notes_dict:
                    notes_dict[element.offset] = []
                notes_dict[element.offset].append(element)
                max_offset = element.offset    
        ret = merge_notes(notes_dict, max_offset)
        #print(len(ret))
        notes = np.append(notes, ret)
        #print(len(notes))
    row = notes.size / 88
    notes = notes.reshape(int(row), 88)
    #with open('data/notes', 'wb') as filepath:
        #pickle.dump(notes, filepath)
    
    return notes

def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 40

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)
    #print(n_patterns)
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 88))
    network_output = np.reshape(network_output, (n_patterns, 88))
    return (network_input, network_output)

def create_network(network_input):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(88))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #model.add(Activation('softmax'))
    #model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def train(model, network_input, network_output, validate_input, validate_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    #print(network_input.shape)
    #print(network_output.shape)
    model.fit(network_input, network_output, 
              validation_data=(validate_input,validate_output), 
              epochs=200, batch_size=64, 
              callbacks=callbacks_list)


In [2]:
#n = get_notes()

In [3]:
train_network()

Parsing midi_songs/Ff7-Cinco.mid
Parsing midi_songs/waltz_de_choco.mid
Parsing midi_songs/Ff4-BattleLust.mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/costadsol.mid
Parsing midi_songs/Ff7-Jenova_Absolute.mid
Parsing midi_songs/tifap.mid
Parsing midi_songs/Still_Alive-1.mid
Parsing midi_songs/bcm.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/ultros.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/mining.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/FFIX_Piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/ff1battp.mid
Parsing midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/ff4-fight1.mid
Parsing midi_songs/Finalfantasy5gilgameshp.mid
Parsing midi_songs/FF4.mid
Parsing midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing midi_songs

Epoch 35/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0172 - acc: 0.9940 - val_loss: 0.0641 - val_acc: 0.9906
Epoch 36/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0164 - acc: 0.9943 - val_loss: 0.0679 - val_acc: 0.9905
Epoch 37/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0159 - acc: 0.9944 - val_loss: 0.0631 - val_acc: 0.9906
Epoch 38/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0153 - acc: 0.9946 - val_loss: 0.0670 - val_acc: 0.9906
Epoch 39/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0149 - acc: 0.9948 - val_loss: 0.0690 - val_acc: 0.9909
Epoch 40/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0144 - acc: 0.9949 - val_loss: 0.0731 - val_acc: 0.9907
Epoch 41/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0139 - acc: 0.9951 - val_loss: 0.0730 - val_acc: 0.9898
Epoch 42/200
46370/46370 [=

Epoch 94/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0053 - acc: 0.9982 - val_loss: 0.0839 - val_acc: 0.9901
Epoch 95/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0052 - acc: 0.9982 - val_loss: 0.0928 - val_acc: 0.9902
Epoch 96/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0052 - acc: 0.9982 - val_loss: 0.0923 - val_acc: 0.9907
Epoch 97/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0052 - acc: 0.9982 - val_loss: 0.0870 - val_acc: 0.9905
Epoch 98/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0050 - acc: 0.9983 - val_loss: 0.0902 - val_acc: 0.9899
Epoch 99/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0050 - acc: 0.9983 - val_loss: 0.0955 - val_acc: 0.9902
Epoch 100/200
46370/46370 [==============================] - 74s 2ms/step - loss: 0.0050 - acc: 0.9983 - val_loss: 0.0915 - val_acc: 0.9900
Epoch 101/200
46370/46370 

In [37]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy as np
from music21 import instrument, note, stream, chord, pitch
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

sequence_length = 40
def get_notes(folder="chopin"):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = np.array([])

    for file in glob.glob("{}/*.mid".format(folder)):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        notes_dict = {}
        max_offset = 0
        for element in notes_to_parse:        
            if isinstance(element, note.Note) or isinstance(element, chord.Chord):  
                if element.offset not in notes_dict:
                    notes_dict[element.offset] = []
                notes_dict[element.offset].append(element)
                max_offset = element.offset    
        ret = merge_notes(notes_dict, max_offset)
        #print(len(ret))
        notes = np.append(notes, ret)
        #print(len(notes))
    row = notes.size / 88
    notes = notes.reshape(int(row), 88)
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    #with open('data/notes', 'rb') as filepath:
    #    notes = pickle.load(filepath)
    
    network_input = prepare_sequences(get_notes())
    for i in range(5):
        # pick a random sequence from the input as a starting point for the prediction
        start = np.random.randint(0, len(network_input)-1)

        arr = {1,50,100,152,191}
        for i in arr:
            model = create_network(i)
            prediction_output = generate_notes(model, network_input, start)
            create_midi(prediction_output, i)

def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    
    network_input = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append(sequence_in)

    return network_input

def create_network(i):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(sequence_length, 88),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(88))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop')
    
    # Load the weights to each node
    if i == 1:
        model.load_weights('weights-improvement-01-0.0808-bigger.hdf5')
    elif i == 50:
        model.load_weights('weights-improvement-50-0.0109-bigger.hdf5')
    elif i == 100:
        model.load_weights('weights-improvement-100-0.0050-bigger.hdf5')
    elif i == 152:
        model.load_weights('weights-improvement-152-0.0035-bigger.hdf5')
    else:
        model.load_weights('weights-improvement-191-0.0029-bigger.hdf5')
    return model

def generate_notes(model, network_input, start):
    """ Generate notes from the neural network based on a sequence of notes """
    
    pattern = network_input[start]
    prediction_output = []
    threshold = 0.5
    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, sequence_length, 88))
        prediction = model.predict(prediction_input, verbose=0)[0]
        prediction[prediction > threshold] = 1
        prediction[prediction <= threshold] = 0   
        pattern = np.append(pattern, prediction)[88:]    
        #print(prediction)
        prediction_output.append(prediction)
    return prediction_output

import time
current_time = lambda: int(round(time.time()))

def create_midi(prediction_output, i):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    #print(prediction_output)
    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        indices = np.where(pattern > 0)[0]
        if len(indices) > 0:
            notes = []
            for index in indices:
                n = note.Note()
                p = pitch.Pitch()
                p.midi = index + 21
                n.pitch = p
                n.storedInstrument = instrument.Piano()
                notes.append(n)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
       
        # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='output/40_sentence_test_output_%d_%s.mid' %(i, current_time()))
    

In [38]:
generate()

Parsing chopin/fruehlingsrauschen_format0.mid
Parsing chopin/chpn_op23.mid
Parsing chopin/chpn-p19.mid
Parsing chopin/chpn_op7_2.mid
Parsing chopin/chpn-p18.mid
Parsing chopin/chpn-p24.mid
Parsing chopin/grieg_kobold.mid
Parsing chopin/clementi_opus36_5_2_format0.mid
Parsing chopin/chpn_op7_1.mid
Parsing chopin/gra_esp_4.mid
Parsing chopin/clementi_opus36_1_2_format0.mid
Parsing chopin/clementi_opus36_6_2_format0.mid
Parsing chopin/chpn-p23.mid
Parsing chopin/chpn-p9.mid
Parsing chopin/clementi_opus36_2_2_format0.mid
Parsing chopin/chpn-p8.mid
Parsing chopin/chpn-p22.mid
Parsing chopin/chpn-p20.mid
Parsing chopin/gra_esp_2.mid
Parsing chopin/gra_esp_3.mid
Parsing chopin/grieg_halling.mid
Parsing chopin/chpn-p21.mid
Parsing chopin/chp_op18.mid
Parsing chopin/chpn_op35_4.mid
Parsing chopin/grieg_waechter.mid
Parsing chopin/clementi_opus36_4_3_format0.mid
Parsing chopin/god_chpn_op10_e01_format0.mid
Parsing chopin/chpn_op33_2.mid
Parsing chopin/chp_op31.mid
Parsing chopin/deb_prel.mid
Par